<a href="https://colab.research.google.com/github/eddiesung111/my-backtest-strategies/blob/main/Common_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datetime
!pip install stockdex -U --no-cache-dir

In [ ]:
# Loading packages
import pandas as pd
import plotly.express as px
import datetime
from stockdex import Ticker

**Key assumptions**

*   no transaction cost
*   trading has no impact on the market
*   only single stock type is supported
*   Allow only one open position at a time
*   no missing data in price history

In [ ]:
def data_processing(price):
  price['timestamp'] = pd.to_datetime(price['timestamp'], format='%Y-%m-%d').dt.normalize()
  price = price.drop(columns = ['currency', 'timezone', 'exchangeTimezoneName', 'exchangeName', 'instrumentType'])
  price = price.rename(columns = {'timestamp': 'time'})
  return price

In [ ]:
class Position:
  def __init__(self, open_datetime, open_price, order_type, volume, sl, tp, comment = 0):
    self.open_datetime = open_datetime
    self.open_price = open_price
    self.order_type = order_type
    self.volume = volume
    self.close_datetime = None
    self.close_price = None
    self.profit = None
    self.status = 'open'
    self.sl = sl
    self.tp = tp
    self.comment = comment

  def close_position(self, close_datetime, close_price):
    self.close_datetime = close_datetime
    self.close_price = close_price
    if self.order_type == 'buy':
        self.profit = (self.close_price - self.open_price) / self.open_price * self.volume
    else:
        self.profit = (self.open_price - self.close_price) / self.open_price * self.volume
    self.status = 'closed'

  def _asdict(self):
    dict = {
        'open_datetime': self.open_datetime,
        'open_price': self.open_price,
        'order_type': self.order_type,
        'volume': self.volume,
        'sl': self.sl,
        'tp': self.tp,
        'close_datetime': self.close_datetime,
        'close_price': self.close_price,
        'profit': self.profit,
        'status': self.status
    }
    return dict


**Data Preprocessing**

Tidy up the data and remove all the useless infomation.


In [ ]:
option = ['AAPL', 'TSLA', 'GE', "AXP", "CSCO", "IBM", "NVDA"]
ticker = Ticker(ticker= option[]) # Try different ticker and see what happens
price = ticker.yahoo_api_price(range='10y', dataGranularity='1d')
price = data_processing(price)
price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2517 entries, 0 to 2516
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   time    2517 non-null   datetime64[ns]
 1   volume  2517 non-null   int64         
 2   close   2517 non-null   float64       
 3   open    2517 non-null   float64       
 4   high    2517 non-null   float64       
 5   low     2517 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 118.1 KB


# 1. SMA crossover algorithm
*   BUY when fast_ma overtakes slow_ma
*   SELL when slow_ma overtakes fast_ma



In [ ]:
# settings
slow_ma = 180
fast_ma = 45

In [ ]:
class MA:
  def __init__(self, data, starting_balance, volume):
    self.position = []
    self.data = data
    self.starting_balance = starting_balance
    self.volume = volume

  def add_positions(self, position):
    self.position.append(position)

  def check_positions(self):
    df = pd.DataFrame(pos._asdict() for pos in self.position)
    if df.loc[df.index[-1], 'status'] == 'open':
      df.loc[df.index[-1], 'close_price'] = self.data.iloc[-1]['close']
      df.loc[df.index[-1], 'profit'] = df.loc[df.index[-1], 'volume'] * (df.loc[df.index[-1], 'close_price'] - df.loc[df.index[-1], 'open_price'])/df.loc[df.index[-1], 'open_price']
    df['pnl'] = df['profit'].cumsum() + self.starting_balance
    return df

  def close_tp_sl(self, data):
    for pos in self.position:
      if pos.status == 'open':
        if (pos.sl >= data.close and pos.order_type == 'buy'):
          pos.close_position(data.time, data.sl)
        elif (pos.sl <= data.close and pos.order_type == 'sell'):
          pos.close_position(data.time, data.sl)
        elif (pos.tp <= data.close and pos.order_type == 'buy'):
          pos.close_position(data.time, data.tp)
        elif (pos.tp >= data.close and pos.order_type == 'sell'):
          pos.close_position(data.time, data.tp)

  def has_open_position(self):
    for pos in self.position:
      if pos.status == 'open':
        return True
    return False

  def run(self):
    for _, row in self.data.iterrows():
      if row.crossover == 'bullish':
        self.add_positions(Position(row.time, row.close, 'buy', self.volume,0 , 0))

      elif row.crossover == 'bearish':
        for pos in self.position:
          if pos.status == 'open':
            pos.close_position(row.time, row.close)
    return self.check_positions()

In [ ]:
# build up SMA columns
price_sma = price.copy()
price_sma['slow_sma'] = price_sma['close'].rolling(window = slow_ma).mean()
price_sma['fast_sma'] = price_sma['close'].rolling(window = fast_ma).mean()

# finding crossovers
price_sma['prev_slow_sma'] = price_sma['slow_sma'].shift(1)
price_sma['prev_fast_sma'] = price_sma['fast_sma'].shift(1)

price_sma.dropna(inplace = True)

def sma_crossover(slow_sma, fast_sma, prev_slow_sma, prev_fast_sma):
  if prev_slow_sma < prev_fast_sma and slow_sma > fast_sma:
    return "bearish"
  elif prev_slow_sma > prev_fast_sma and slow_sma < fast_sma:
    return "bullish"
  return False

price_sma['crossover'] = price_sma.apply(lambda x: sma_crossover(x['slow_sma'], x['fast_sma'], x['prev_slow_sma'], x['prev_fast_sma']), axis = 1)

price_sma_cross = price_sma[price_sma['crossover'] != False]

In [ ]:
fig = px.line(price_sma, x = 'time', y = ['close', 'slow_sma', 'fast_sma'])

for i, row in price_sma_cross.iterrows():
  if row.crossover == 'bearish':
    fig.add_vline(x = row.time, line_color="red", line_dash="dash")
  else:
    fig.add_vline(x = row.time, line_color="green", line_dash="dash")
fig.show()

In [ ]:
sma_action = MA(price_sma, 10000, 10000)
sma_pos = sma_action.run()
sma_pos

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2015-10-29,140.610107,buy,10000,0,0,2016-10-11,138.597290,-143.148840,closed,9856.851160
1,2016-12-27,152.878754,buy,10000,0,0,2017-02-28,142.862564,-655.172111,closed,9201.679049
2,2019-05-03,52.333416,buy,10000,0,0,2019-09-05,43.910229,-1609.523685,closed,7592.155363
3,2019-11-22,57.566757,buy,10000,0,0,2020-04-02,34.390530,-4025.974138,closed,3566.181226
4,2020-11-11,44.308960,buy,10000,0,0,2021-10-01,65.927643,4879.077026,closed,8445.258252
5,2022-11-30,53.560760,buy,10000,0,0,NaT,168.369995,21435.325703,open,29880.583955


In [ ]:
fig_sma = px.line(sma_pos, x = 'open_datetime', y = 'pnl')
fig_sma

# 2. EMA crossover algorithm

In [ ]:
price_ema = price.copy()

# Calculate EMAs
price_ema['slow_ema'] = price_ema['close'].ewm(span=slow_ma, adjust=False).mean()
price_ema['fast_ema'] = price_ema['close'].ewm(span=fast_ma, adjust=False).mean()

# Finding crossovers
price_ema['prev_slow_ema'] = price_ema['slow_ema'].shift(1)
price_ema['prev_fast_ema'] = price_ema['fast_ema'].shift(1)

price_ema.dropna(inplace=True)

def ema_crossover(slow_ema, fast_ema, prev_slow_ema, prev_fast_ema):
  if prev_slow_ema < prev_fast_ema and slow_ema > fast_ema:
    return "bearish"
  elif prev_slow_ema > prev_fast_ema and slow_ema < fast_ema:
    return "bullish"
  return False

price_ema['crossover'] = price_ema.apply(lambda x: ema_crossover(x['slow_ema'], x['fast_ema'], x['prev_slow_ema'], x['prev_fast_ema']), axis=1)

price_ema_cross = price_ema[price_ema['crossover'] != False]
#price_ema_cross

In [ ]:
fig = px.line(price_ema, x = 'time', y = ['close', 'slow_ema', 'fast_ema'])

for i, row in price_ema_cross.iterrows():
  if row.crossover == 'bearish':
    fig.add_vline(x = row.time, line_color="red", line_dash="dash")
  else:
    fig.add_vline(x = row.time, line_color="green", line_dash="dash")

fig.show()

In [ ]:
action = MA(price_ema, 10000, 10000)
ema_pos = action.run()
ema_pos

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2015-04-13,132.415039,buy,10000,0,0,2015-08-24,114.395477,-1360.839516,closed,8639.160484
1,2015-10-14,132.271271,buy,10000,0,0,2016-10-10,138.309738,456.521463,closed,9095.681947
2,2016-11-29,148.805176,buy,10000,0,0,2017-02-28,142.862564,-399.355175,closed,8696.326773
3,2019-07-30,52.433098,buy,10000,0,0,2019-08-02,49.841351,-494.296044,closed,8202.030729
4,2019-11-13,56.270882,buy,10000,0,0,2020-03-17,35.287674,-3728.963735,closed,4473.066994
5,2020-11-16,47.698170,buy,10000,0,0,2021-12-15,57.367393,2027.168725,closed,6500.235718
6,2022-11-25,54.912704,buy,10000,0,0,NaT,168.369995,20661.391885,open,27161.627604


In [ ]:
fig_ema = px.line(ema_pos, x = 'open_datetime', y = 'pnl')
fig_ema

# 3. RSI (Relative Strength Index)
*   BUY if RSI indicator < 20
*   SELL if RSI indicator > 80




In [ ]:
rsi_period = 20
price_rsi = price.copy()

price_rsi['gain'] = (price_rsi['close'] - price_rsi['open']).apply(lambda x: x if x > 0 else 0)
price_rsi['loss'] = (price_rsi['close'] - price_rsi['open']).apply(lambda x: -x if x < 0 else 0)

price_rsi['ema_gain'] = price_rsi['gain'].ewm(span = rsi_period, min_periods = rsi_period).mean()
price_rsi['ema_loss'] = price_rsi['loss'].ewm(span = rsi_period, min_periods = rsi_period).mean()

price_rsi['rs'] = price_rsi['ema_gain'] / price_rsi['ema_loss']
price_rsi['rsi_14'] = 100 - (100 / (1 + price_rsi['rs'])) # This line creates the 'rsi_14' column

price_rsi['prev_rsi'] = price_rsi['rsi_14'].shift(1) # Now this line can access 'rsi_14' without error

atr_period = 20

price_rsi['range'] = price_rsi['high'] - price_rsi['low']
price_rsi['atr_14'] = price_rsi['range'].rolling(window = atr_period).mean()

price_rsi.dropna(inplace = True)

def rsi_strategy(prev_rsi, rsi):
  if prev_rsi < 80 and rsi > 80:
    return "sell"
  elif prev_rsi > 20 and rsi < 20:
    return "buy"
  return False

price_rsi['action'] = price_rsi.apply(lambda x: rsi_strategy(x['prev_rsi'], x['rsi_14']), axis=1)

price_rsi_action = price_rsi[price_rsi['action'] != False]

In [ ]:
fig = px.line(price_rsi, x = 'time', y = 'close')


for i, row in price_rsi_action.iterrows():
  if row.action == 'sell':
    fig.add_vline(x = row.time, line_color="red", line_dash="dash")
  else:
    fig.add_vline(x = row.time, line_color="green", line_dash="dash")

fig.show()


In [ ]:
class RSI:
  def __init__(self, df, starting_balance, volume):
    self.positions = []
    self.data = df
    self.starting_balance = starting_balance
    self.volume = volume

  def add_positions(self, position):
    self.positions.append(position)
    return True

  def check_positions(self):
    df = pd.DataFrame(pos._asdict() for pos in self.positions)
    if df.loc[df.index[-1], 'status'] == 'open':
      df.loc[df.index[-1], 'close_price'] = self.data.iloc[-1]['close']
      df.loc[df.index[-1], 'profit'] = df.loc[df.index[-1], 'volume'] * (df.loc[df.index[-1], 'close_price'] - df.loc[df.index[-1], 'open_price'])/df.loc[df.index[-1], 'open_price']
    df['pnl'] = df['profit'].cumsum() + self.starting_balance
    return df

  def close_tp_sl(self, data):
    for pos in self.positions:
      if pos.status == 'open':
        if (pos.sl >= data.close and pos.order_type == 'buy'):
          pos.close_position(data.time, pos.sl)
        elif (pos.sl <= data.close and pos.order_type == 'sell'):
          pos.close_position(data.time, pos.sl)
        elif (pos.tp <= data.close and pos.order_type == 'buy'):
          pos.close_position(data.time, pos.tp)
        elif (pos.tp >= data.close and pos.order_type == 'sell'):
          pos.close_position(data.time, pos.tp)

  def has_open_position(self):
    for pos in self.positions:
      if pos.status == 'open':
        return True
    return False

  def logic(self, data):
    if not self.has_open_position():
      if data['rsi_14'] < 20:
        datatime = data.time
        open_price = data.close
        order_type = 'buy'
        volume = self.volume
        sl = open_price - 2 * data['atr_14']
        tp = open_price + 2 * data['atr_14']
        self.add_positions(Position(datatime, open_price, order_type, volume, sl, tp))

      elif data['rsi_14'] > 80:
        datatime = data.time
        open_price = data.close
        order_type = 'sell'
        volume = self.volume
        tp = open_price - 2 * data['atr_14']
        sl = open_price + 2 * data['atr_14']

        self.add_positions(Position(datatime, open_price, order_type, volume, sl, tp))

  def run(self):
    for _, row in self.data.iterrows():
      self.close_tp_sl(row)
      self.logic(row)

    return self.check_positions()

In [ ]:
rsi_action = RSI(price_rsi, 10000, 10000)
rsi_pos = rsi_action.run()
rsi_pos

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2015-01-28,114.251709,buy,10000,110.139800,118.363618,2015-02-10,118.363618,359.899100,closed,10359.899100
1,2015-03-02,125.130539,sell,10000,127.996415,122.264663,2015-03-06,122.264663,229.030901,closed,10588.930001
2,2015-04-10,136.632385,sell,10000,140.931203,132.333568,2015-04-15,132.333568,314.626539,closed,10903.556539
3,2015-10-08,134.332016,sell,10000,139.488676,129.175356,2015-10-22,139.488676,-383.874270,closed,10519.682269
4,2015-10-22,141.760300,sell,10000,146.811529,136.709071,2015-11-11,146.811529,-356.321841,closed,10163.360428
5,2015-11-11,146.984055,sell,10000,152.054451,141.913658,2016-01-07,141.913658,344.962346,closed,10508.322774
6,2016-03-28,150.913849,sell,10000,154.656744,147.170953,2016-04-07,147.170953,248.015377,closed,10756.338151
7,2016-06-16,146.840286,sell,10000,149.979333,143.701239,2016-06-24,143.701239,213.772890,closed,10970.111041
8,2016-07-06,152.111969,sell,10000,156.223880,148.000058,2016-07-14,156.223880,-270.321332,closed,10699.789709
9,2016-07-14,156.377228,sell,10000,160.168044,152.586412,2016-07-25,152.586412,242.414819,closed,10942.204528


In [ ]:
rsi_pos = rsi_pos[rsi_pos['status'] == 'closed']
fig_rsi = px.line(rsi_pos, x = 'open_datetime', y = 'pnl')
fig_rsi

# 4. Buy-Hold Strategy
* BUY and Hold when drawdown reaches 5%, 15%, 35%
* SELL when price reach previous all-time-high

In [ ]:
price_bh = price.copy()
price_bh['all-time-high'] = price_bh['close'].expanding().max()
price_bh['drawdown'] = 1 - price_bh['close'] / price_bh['all-time-high']

In [ ]:
# BUY and Hold when drawdown reaches 5%, 15%, 35%
# SELL when price reach previous all-time-high

def get_signal(row):
    drawdown = row['drawdown']

    if drawdown >= 0.35:
        return '>35% dd'
    elif drawdown >= 0.15:
        return '>15% dd'
    elif drawdown >= 0.05:
        return '>5% dd'
    else:
        return '< 5% dd'

price_bh['signal'] = price_bh.apply(get_signal, axis=1)
display(px.scatter(price_bh, x='time', y='drawdown', color='signal'))

price_bh['count_value'] = 1
display(px.pie(price_bh.groupby('signal').agg({'count_value': 'count'}).reset_index(), values='count_value', names='signal'))

In [ ]:
class BH:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        self.trading_allowed = True

    def check_positions(self):
      df = pd.DataFrame(pos._asdict() for pos in self.positions)

      for idx in df.index:
            if df.loc[idx, 'status'] == 'open':
                matching_row = self.data[self.data['time'] == df.loc[idx, 'open_datetime']].iloc[0]

                df.loc[idx, 'close_price'] = self.data.loc[idx, 'close']
                if df.loc[idx, 'order_type'] == 'buy':
                    df.loc[idx, 'profit'] = df.loc[idx, 'volume'] * (df.loc[idx, 'close_price'] - df.loc[idx, 'open_price']) / df.loc[idx, 'open_price']
                else:
                    df.loc[idx, 'profit'] = df.loc[idx, 'volume'] * (df.loc[idx, 'open_price'] - df.loc[idx, 'close_price']) / df.loc[idx, 'open_price']
      df['pnl'] = df['profit'].cumsum() + self.starting_balance
      return df

    def add_position(self, position):
        self.positions.append(position)
        return True

    def trade(self, drawdown, data):
        self.trading_allowed = True
        if data.signal == drawdown:
            for pos in self.positions:
                if pos.status == 'open' and pos.comment == drawdown:
                    self.trading_allowed = False
                    break
            if self.trading_allowed:
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 0.0, 0.0, drawdown))


    def run(self):
        for i, data in self.data.iterrows():

            # opening positions at different dd levels
            self.trade('>5% dd', data)
            self.trade('>15% dd', data)
            self.trade('>35% dd', data)

            # if drawdown is 0, close all positions
            if data.drawdown == 0.0:
                for pos in self.positions:
                    if pos.status == 'open':
                        pos.close_position(data.time, data.close)

        return self.check_positions()

In [ ]:
bh_action = BH(price_bh, 10000, 500)
bh_pos = bh_action.run()
fig_bh = px.line(bh_pos, x='open_datetime', y='pnl')

display(bh_pos)
display(fig_bh)

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2015-01-06,115.353966,buy,500,0.0,0.0,2015-02-25,124.172050,38.221853,closed,10038.221853
1,2015-03-26,118.852448,buy,500,0.0,0.0,2015-04-10,136.632385,74.798366,closed,10113.020219
2,2015-04-20,129.491653,buy,500,0.0,0.0,2015-10-16,138.884842,36.269475,closed,10149.289694
3,2015-08-24,114.395477,buy,500,0.0,0.0,2015-10-16,138.884842,107.038168,closed,10256.327862
4,2016-01-07,138.836914,buy,500,0.0,0.0,2016-03-28,150.913849,43.493241,closed,10299.821103
5,2016-05-04,144.108597,buy,500,0.0,0.0,2016-07-08,154.316483,35.417338,closed,10335.238441
6,2016-07-27,149.907440,buy,500,0.0,0.0,2024-04-23,162.619995,42.401348,closed,10377.639789
7,2017-05-17,131.360703,buy,500,0.0,0.0,2024-04-23,162.619995,118.982664,closed,10496.622452
8,2017-10-26,102.174767,buy,500,0.0,0.0,2024-04-23,162.619995,295.793329,closed,10792.415781
9,2024-05-13,159.500000,buy,500,0.0,0.0,2024-07-23,172.000000,39.184953,closed,10831.600734


In [ ]:
fig = px.line(price_bh, x='time', y=['close', 'all-time-high'], title='DAX - Buy Drawdown Strategy')

# adding trades to plots
for i, position in bh_pos.iterrows():
    if position.status == 'closed':
        fig.add_shape(type="line",
            x0=position.open_datetime, y0=position.open_price, x1=position.close_datetime, y1=position.close_price,
            line=dict(
                color="green" if position.profit >= 0 else "red",
                width=3)
            )
fig

# 5. Buy and Hold
*   Buy at the start and do nothing



In [ ]:
class BuyAndHold:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df

    def check_positions(self):
        df = pd.DataFrame(pos._asdict() for pos in self.positions)
        if not df.empty and df.loc[df.index[-1], 'status'] == 'open':
            df.loc[df.index[-1], 'close_price'] = self.data.iloc[-1]['close']
            df.loc[df.index[-1], 'profit'] = df.loc[df.index[-1], 'volume'] * (df.loc[df.index[-1], 'close_price'] - df.loc[df.index[-1], 'open_price']) / df.loc[df.index[-1], 'open_price']
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df

    def add_position(self, position):
        self.positions.append(position)
        return True

    def run(self):
        # Buy at the beginning
        self.add_position(Position(self.data.iloc[0]['time'], self.data.iloc[0]['close'], 'buy', self.volume, 0.0, 0.0))
        # Hold until the end
        return self.check_positions()

buy_and_hold_action = BuyAndHold(price, 10000, 10000)
buy_and_hold_pos = buy_and_hold_action.run()
buy_and_hold_pos

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2014-12-22,123.213562,buy,10000,0.0,0.0,None,168.369995,3664.891459,open,13664.891459


# Conclusion and Visualization

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

# Buy-Hold Strategy
fig.add_trace(go.Scatter(x=bh_pos['open_datetime'], y=bh_pos['pnl'], mode='lines', name='Buy-Hold[All time High strategy]'))

# SMA Crossover Strategy
fig.add_trace(go.Scatter(x=sma_pos['open_datetime'], y=sma_pos['pnl'], mode='lines', name='SMA Crossover'))

# EMA Crossover Strategy
fig.add_trace(go.Scatter(x=ema_pos['open_datetime'], y=ema_pos['pnl'], mode='lines', name='EMA Crossover'))

# RSI Strategy
fig.add_trace(go.Scatter(x=rsi_pos['open_datetime'], y=rsi_pos['pnl'], mode='lines', name='RSI Strategy'))
fig.show()

In [ ]:
# Assuming you have already calculated these values:
rsi_pnl = rsi_pos.loc[rsi_pos.index[-1], 'pnl']
bh_pnl = bh_pos.loc[bh_pos.index[-1], 'pnl']
sma_pnl = sma_pos.loc[sma_pos.index[-1], 'pnl']
ema_pnl = ema_pos.loc[ema_pos.index[-1], 'pnl']
bh2_pnl = buy_and_hold_pos.loc[buy_and_hold_pos.index[-1], 'pnl']

def calculate_yearly(pnl):
  result = pnl / 10000
  result = result ** (1/10) - 1
  result = result * 100
  return result

# Calculate yearly returns
rsi_avg = calculate_yearly(rsi_pnl)
bh_avg = calculate_yearly(bh_pnl)
sma_avg = calculate_yearly(sma_pnl)
ema_avg = calculate_yearly(ema_pnl)
bh2_avg = calculate_yearly(bh2_pnl)
spx_avg = 13.21

# Create the DataFrame
data = {'Strategy': ['S&P 500', 'SMA', "EMA", 'RSI', 'Buy-Hold[All-Time-High Strategy]', 'Buy-Hold' ],
        'Yearly Average Return(in %)': [spx_avg, sma_avg, ema_avg, rsi_avg, bh_avg, bh2_avg]}
avg_df = pd.DataFrame(data)

# Display the DataFrame
display(avg_df)

,Strategy,Yearly Average Return(in %)
0,S&P 500,13.210000
1,SMA,11.567810
2,EMA,10.508473
3,RSI,-2.699919
4,Buy-Hold[All-Time-High Strategy],0.568955
5,Buy-Hold,3.171708


It is not fair on using only one strategy over the period of time, but we can see that the performance of RSI and Buy-Hold Strategy is much worse than others.
In addition, ther performance of SMA and EMA sometimes surpass the performance of S&P 500, but most of the time doesn't. Therefore, common most of the common strategy cannot win the market.

For the follow-up projects, I would try to implement DRL to build a stragies to trade and let's see whether it would perform better than the market or not.